### Algorithm for the conversion of the Medibio's ECG file set into .csv files

###### written by Nicola Pastorello 19/0/2015

In [23]:
%matplotlib inline
# Loading libraries
import numpy as np
import pandas as pd
import datetime, pickle
import os, glob, sys
from pylab import *


In [2]:
# Functions
def findCharPos(stringInput, c=' '):
    for ii in np.arange(len(stringInput)):
        if stringInput[ii] == c:
            return ii, stringInput[:ii]
    return False

def removeFChar(stringInput, cc): #Remove extra characters at the end
    for ii in np.arange(len(stringInput)-1, -1, -1):
        if stringInput[ii] != cc:
            return stringInput[:ii+1]

In [16]:
# Loading data and saving as csv files

if not(os.path.exists('Fileset0_csv')):
    os.mkdir('Fileset0_csv')

listFiles = glob.glob('Fileset0/*.txt')
#dicTot = {}
listFail = []
for ii in listFiles:
    try:
        fileIn = open(ii, 'rb'); f = fileIn.read(); fileIn.close()
        ID = findCharPos(f, c='\t')[1]
#        # Check if ID also in file name
#        checkID = False
#        if ID in ii: checkID = True
#        if not(checkID): raise ValueError('ID not matching with filename')
            
        # Position of first row
        posFirstRow = findCharPos(f, c=':')[0]-2
            
        # Reading line by line and storing it to list
        listRows, listTmp = ['time,hb,var1,var2'], []
        for jj in np.arange(len(f)-posFirstRow):
            if f[posFirstRow+jj] == '\t':
                listTmp.append(',')
            elif (f[posFirstRow+jj] != '\r') and (f[posFirstRow+jj] != '\n'):
                listTmp.append(f[posFirstRow+jj])
            else:
                totLine = ''.join(listTmp)
                if totLine:
                    listRows.append(removeFChar(totLine, ','))
                    listTmp = [] 
        
        outTab = '\n'.join(listRows)
        #dicTot[ii[9:-4]] = '\n'.join(listRows)
        fileOut = open('Fileset0_csv/'+ii[9:-4]+'.csv', 'wb')
        fileOut.write(outTab)
        fileOut.close()
        
    except:
        ee = sys.exc_info()[0]
        listFail.append([ii, ee])

In [17]:
# Check exceptions
if len(listFail) > 0: 
    print "Errors with the following files:"
    for ii in listFail:
        print "\n"+ii[0]
else:
    print "No errors"

No errors


In [18]:
# Creating pandas dataframe
dicDF = {}
listFail = []
for ii in glob.glob('Fileset0_csv/*.csv'):
    try:
#    ii = glob.glob('Fileset0_csv/*.csv')[3]
        df = pd.read_csv(ii)
#print df

# Removing zeropoint
        tmp = pd.to_datetime(df['time'])-pd.to_datetime(df['time'][0])
        df['time'] = tmp
        dicDF[ii] = df
    except:
        ee = sys.exc_info()[0]
        listFail.append([ii, ee])

In [19]:
# Check exceptions
if len(listFail) > 0: 
    print "Errors with the following files:"
    for ii in listFail:
        print "\n"+ii[0]
else:
    print "No errors"

No errors


In [24]:
# Saving output
fileOut = open('dicDF.dat', 'wb')
pickle.dump(dicDF, fileOut)
fileOut.close()